In [11]:
import pandas as pd
import neuralcoref
import spacy

import nltk
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
 

In [2]:
# baca file dataset
df = pd.read_csv('movies_genres.csv', sep='\t', nrows=100)

# aku kepikirannya bikin model buat setiap genre, jadi labelnya dipisah
label_action = df['Action']
label_adult = df['Adult']
label_adventure = df['Adventure']
label_animation = df['Animation']
label_biography = df['Biography']
label_comedy = df['Comedy']
label_crime = df['Crime']
label_documentary = df['Documentary']
label_drama = df['Drama']
label_family = df['Family']
label_fantasy = df['Fantasy']
label_gameshow = df['Game-Show']
label_history = df['History']
label_horror = df['Horror']
label_lifestyle = df['Lifestyle']
label_music = df['Music']
label_musical = df['Musical']
label_mystery = df['Mystery']
label_news = df['News']
label_reality = df['Reality-TV']
label_romance = df['Romance']
label_scifi = df['Sci-Fi']
label_short = df['Short']
label_sport = df['Sport']
label_talkshow = df['Talk-Show']
label_thriller = df['Thriller']
label_war = df['War']
label_western = df['Western']
df = df.drop(columns=['Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Game-Show', 'History', 'Horror', 'Lifestyle', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western'])

df.head()

,title,plot
0,"""#7DaysLater"" (2013)",#7dayslater is an interactive comedy series f...
1,"""#BlackLove"" (2015) {Crash the Party (#1.9)}","With just one week left in the workshops, the..."
2,"""#BlackLove"" (2015) {Making Lemonade Out of Le...",All of the women start making strides towards...
3,"""#BlackLove"" (2015) {Miss Independent (#1.5)}",All five of these women are independent and s...
4,"""#BlackLove"" (2015) {Sealing the Deal (#1.10)}",Despite having gone through a life changing p...


In [10]:
# coreference resolution

# experiment
text = """

Walt Disney was born on December 5, 1901. Disney became one of the best-known motion picture producers in the world. He is particularly noted for being a film producer and a popular showman, as well as an innovator in animation and theme park design.

Disney is famous for his contributions in the field of entertainment during the 20th century. His first success was through the series, Oswald the Lucky Rabbit which was created by the Disney studio for Charles Mintz of Universal Studios. When Disney asked for a larger budget for his popular Oswald series, Mintz refused and Disney had to quit. Later, Disney and his brother Roy O. Disney started from scratch and co-founded Walt Disney Productions, now known as The Walt Disney Company. Today, this company has annual revenues of approximately U.S. $35 billion. This success is largely due to a number of the world's most famous fictional characters he and his staff created including Mickey Mouse, a character for which Disney himself was the original voice.

Disney won 26 Academy Awards out of 59 nominations, including a record four in one year, giving him more awards and nominations than any other individual. He is also the namesake for Disneyland and Walt Disney World Resort theme parks in the United States, as well as the international resorts in Japan, France, and China.

Disney died of lung cancer in Burbank, California, on December 15, 1966. The following year, construction began on Walt Disney World Resort in Florida. His brother Roy Disney inaugurated The Magic Kingdom on October 1, 1971.

"""

nlp = spacy.load('en_core_web_sm')
neuralcoref.add_to_pipe(nlp, greedyness=0.45, max_dist=40, max_dist_match=500)
doc = nlp(text)

print(doc._.coref_clusters)
print(doc._.coref_resolved)

# pre-processing
def coref_resolution(dataframe, nlp):
    for i in range(0, len(dataframe)):
        doc = nlp(dataframe['plot'][i])
        dataframe.at[i, 'plot'] = doc._.coref_resolved
    
coref_resolution(df, nlp)

[

Walt Disney: [

Walt Disney, Disney, Disney, Disney, Disney, Disney, Disney, Disney, Disney himself, Disney, Disney], He: [He, his, His, his], Charles Mintz: [Charles Mintz, Mintz, his], The Walt Disney Company: [The Walt Disney Company, this company], his: [his, him, He]]


Walt Disney was born on December 5, 1901. 

Walt Disney became one of the best-known motion picture producers in the world. He is particularly noted for being a film producer and a popular showman, as well as an innovator in animation and theme park design.



Walt Disney is famous for He contributions in the field of entertainment during the 20th century. He first success was through the series, Oswald the Lucky Rabbit which was created by the 

Walt Disney studio for Charles Mintz of Universal Studios. When 

Walt Disney asked for a larger budget for He popular Oswald series, Charles Mintz refused and 

Walt Disney had to quit. Later, 

Walt Disney and Charles Mintz brother Roy O. Disney started from scratch a

In [9]:
# lemmatization
df.head()

,title,plot
0,"""#7DaysLater"" (2013)",#7dayslater is an interactive comedy series f...
1,"""#BlackLove"" (2015) {Crash the Party (#1.9)}","With just one week left in the workshops, the..."
2,"""#BlackLove"" (2015) {Making Lemonade Out of Le...",All of the women start making strides towards...
3,"""#BlackLove"" (2015) {Miss Independent (#1.5)}",All five of these women are independent and s...
4,"""#BlackLove"" (2015) {Sealing the Deal (#1.10)}",Despite having gone through a life changing p...


In [15]:
# pos tag
def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
    }

def untag(tagged_sentence):
    return [w for w, t in tagged_sentence]

tagged_sentences = nltk.corpus.treebank.tagged_sents()

# Split the dataset for training and testing
cutoff = int(.75 * len(tagged_sentences))
training_sentences = tagged_sentences[:cutoff]
test_sentences = tagged_sentences[cutoff:]
 
print(len(training_sentences))   # 2935
print(len(test_sentences))    # 979
 
def transform_to_dataset(tagged_sentences):
    X, y = [], []
 
    for tagged in tagged_sentences:
        for index in range(len(tagged)):
            X.append(features(untag(tagged), index))
            y.append(tagged[index][1])
 
    return X, y
 
X, y = transform_to_dataset(training_sentences)

clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', DecisionTreeClassifier(criterion='entropy'))
])
 
clf.fit(X[:10000], y[:10000])   # Use only the first 10K samples if you're running it multiple times. It takes a fair bit :)
 
print('Training completed')
 
X_test, y_test = transform_to_dataset(test_sentences)
 
print("Accuracy:", clf.score(X_test, y_test))

2935
979
Training completed
Accuracy: 0.896111200385666


In [6]:
# ner

In [7]:
# event extraction

In [8]:
# main program (learning)